In [ ]:
import pypsa
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

## Principal Flow Patterns

In [ ]:
X = n.buses_t.marginal_price.loc[:, n.buses.carrier == "AC"]

In [ ]:
balance = nodal_balance(n, "AC")
X = -balance.droplevel(0).unstack("carrier")[["AC", "DC"]].sum(axis=1).unstack(level=0)

In [ ]:
sc = StandardScaler(with_std=False)
X_std = sc.fit_transform(X)

In [ ]:
pca = PCA()
X_pca = pca.fit_transform(X_std)

In [ ]:
eigvec_std = pd.DataFrame(sc.inverse_transform(pca.components_), columns=X.columns).T

In [ ]:
eigvec = pd.DataFrame(pca.components_, columns=X.columns).T

In [ ]:
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel("number of components")
plt.ylabel("cumulative explained variance");

In [ ]:
eigval = pd.Series(pca.explained_variance_ratio_)

In [ ]:
(eigvec_std * eigval).sum(axis=1)  # check that gives X.sum()

In [ ]:
proj = ccrs.PlateCarree()
cmap = "RdBu"
label = "test"

fig, axes = plt.subplots(1, 4, figsize=(20, 7), subplot_kw={"projection": proj})

kwargs = dict(
    transform=ccrs.PlateCarree(),
    cmap=cmap,
    linewidths=0,
    vmin=-0.5,
    vmax=0.5,
)

legend_kwargs = dict(
    legend=True,
    legend_kwds={
        "label": label,
        "shrink": 0.7,
        "extend": "max",
    },
)

values = eigvec
# values = eigvec_std * eigval

for i, ax in enumerate(axes):

    # if i == len(axes) - 1:
    kwargs.update(legend_kwargs)

    nodes.plot(ax=ax, column=values[i], **kwargs)

    ax.set_title(rf"$\lambda_{i} = {eigval[i]:.2f}$")

    ax.add_feature(cartopy.feature.COASTLINE.with_scale("50m"), linewidth=0.2, zorder=2)
    ax.add_feature(cartopy.feature.BORDERS.with_scale("50m"), linewidth=0.2, zorder=2)

    ax.outline_patch.set_visible(False)
    # plt.gca().outline_patch.set_visible(False)

plt.tight_layout()

plt.savefig("test.pdf", bbox_inches="tight")

In [ ]:
crs = ccrs.EuroPP()

fig, ax = plt.subplots(figsize=(7, 6), subplot_kw={"projection": crs})

n.plot(
    branch_components=["Line"],
    ax=ax,
    geomap=True,
    flow=pd.concat({"Line": vectors[0] * 1e3}),
)

plt.savefig("test.pdf", bbox_inches="tight")